# Wizard Crawler 
*(noun)*
1. Crawls threads in search of comments from the wizard, /u/XiuathoTheWizard, and compiles them into a .json file.
2. Converts the result into an easy to navigate html.
3. For more info see the [Github repository](https://github.com/XkF21WNJ/WizardTree)

Credits to /u/XiuathoTheWizard and everyone else who contributed to the story.

Code by /u/XkF21WNJ.

### Crawl Thread

In [ ]:
#Imports
import praw
import markdown as md

#Definitions
def YieldAll(comments):
    for c in comments:
        if isinstance(c, praw.objects.MoreComments):
            for x in YieldAll(c.comments()): yield x
        else: yield c

def IsWizard(story, comment):
    return str(comment.author) == story['author']

def FindWizard(story, comments):
    for c in YieldAll(comments):
        if IsWizard(story, c): yield c

def GetEvent(story, comments, parent=""):
    for c in FindWizard(story, comments):
        story['events'][c.id] = {"description": md.markdown(c.body), 
                                 "actions" : list(GetActions(story, c)),
                                 "parent" : parent,
                                 "url" : c.permalink}
        if parent == "": story['start'] = c.id
        return c.id

def GetActions(story, comment):
    for r in YieldAll(comment.replies):
        consequence = GetEvent(story, r.replies, comment.id)
        if consequence == None: continue;
        story['actions'][r.id] = {"description": md.markdown(r.body),
                                  "consequence" : consequence,
                                  "actor" : r.author.name,
                                  "url" : r.permalink}
        story['actors'].append(r.author.name)
        yield r.id;
        
def Crawl(submission_id, author):
    #Initialisation
    r = praw.Reddit(user_agent='WizardCrawler/0.1 (by /u/XkF21WNJ)')
    submission = r.get_submission(submission_id = submission_id)    
    story = {'author' : author, 
             'url' : submission.url,
             'start' : "",
             'events' : {}, 
             'actions' : {}, 
             'actors' : []}

    #Parse thread
    GetEvent(story, submission.comments)
    
    #Finalize
    uniq_actors = list(set(story['actors']))
    uniq_actors.sort(key = story['actors'].count, reverse = True)
    story['actors'] = uniq_actors
    
    return story

story = Crawl('3xyesc', "XiuathoTheWizard")

### Import / Export

#### Import tree from file

In [1]:
import json

f = open("Output/WizardTree.json",'r')
story = json.load(f)
f.close()

#### Export tree to file

In [75]:
import json

f = open("Output/WizardTree.json",'w')
json.dump(story, f, indent=4)
f.close()

### HTML conversion

In [2]:
#Imports
from html import HTML

#Definitions
def Truncate(text):
    return text[:60] + (text[60:] and '..')

def ParseEvent(story, body, event_id, index):
    event = story['events'][event_id]
    div = body.div(klass = 'event')
    div.a('', name = event_id)
    
    links = div.div(klass = 'link-container')
    if event['parent'] != None:     
        links.a('Go to Previous', 
                href = '#'+event['parent'],
                klass = 'left-link')
    else:
        links.a('Go to Index', href = '#index', klass = 'left-link')
    links.a('Go to Original', href = event['url'], klass = 'right-link')
    
    div.text(event['description'], escape=False)
    
    for action_id in event['actions']:
        ParseAction(story, body, action_id, div.ul(), index)

def ParseAction(story, body, action_id, ul, index):
    action = story['actions'][action_id]
    href = '#'+action['consequence']
    
    indli = index.li()
    indli.a(action['description'], href = href, klass='action', style = 'display: block;', escape=False)
    
    li = ul.li(klass = 'action')
    a = li.a(href = href,
             style = "display: block;", 
             title = '/u/'+ action['actor'])
    
    a.span(action['description'], escape=False)
    ParseEvent(story, body, action['consequence'], index)

def ParseStory(story):    
    #Styling
    css = """
        body {
            font-family: arial, sans-serif;
            width: 40em;
            margin-left:auto;
            margin-right:auto;
        }
        
        p {
            text-align: justiy;
            -moz-hyphens: auto;
            hyphens: auto;
        }
        
        .event {
            margin-bottom: 1000ex;
        }
        
        .event > p {
            font-family: Georgia, serif;
            font-size: larger;
            line-height: 1.5em;
        }
        
        .action p {
            margin-top: 0.25em;
            margin-bottom: 0.25em;
        }
        
        .action {
            font-family: consolas, monospace;
            margin: 0 0 2ex 0;
        }
        
        .link-container {
            font-family: consolas, monospace;
            display: flex;
        }
        
        .left-link{
            width: 40ex;
        }
        
        .right-link{
            flex-grow: 1;
            text-align: right;
        }
    """

    #Build HTML file
    doc = HTML()
    doc.text("<!DOCTYPE html>", escape=False)

    html = doc.html()
    header = html.head()
    header.meta(charset = "UTF-8")
    header.title("XiuathoTheWizard's Epic Quest.")
    header.style(css, escape = False)

    body = html.body()
    index = HTML().ul()

    #Parse tree
    ParseEvent(story, body, story['start'], index)

    #Add index
    body.a('', name = 'index')
    body.h1('Index')
    body.div(index, escape = False)

    #Add credits
    body.a('', name = 'credits')
    body.h1('Credits')
    body.p("A special thanks to /u/XiuathoTheWizard for creating the story, and bringing the great folks at /r/TalesFromTechSupport a tale that won't soon be forgotten.")
    body.p("Also thanks to all the commenters appearing in the story:")
    users_ul = body.ul()
    for actor in story['actors']: users_ul.li("/u/"+actor)

    body.p("Powered by python, coded by /u/XkF21WNJ.")
    
    return doc
    
doc = ParseStory(story)

#Export to file
f = open("Output/WizardTree.html", 'w')
f.write(unicode(doc).encode('utf-8'))
f.close()